# Credit Risk Prediction: Data Cleaning & Preprocessing

This notebook is part of a larger project exploring machine learning approaches to **credit risk prediction** using real-world LendingClub loan data (2007–2018). The goal is to build predictive models that can assess the likelihood of a loan default based on borrower attributes and financial history.

> The cleaned dataset produced here will be used in subsequent notebooks to train and evaluate various machine learning models including logistic regression, decision trees, random forests, and gradient boosting.

---


## Load the LendingClub Dataset

Importing the LendingClub "Accepted Loans (2007–2018Q4)" dataset, which contains millions of rows and a wide variety of loan-related features. Since it's a large file, we used `low_memory=False` to ensure all data types are properly inferred without memory fragmentation.

In [1]:
import pandas as pd

# Load the data (this might take a moment – it's large)
df = pd.read_csv('../data/accepted_2007_to_2018Q4.csv', low_memory=False)

# Show first rows
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## Filter for Fully Paid and Charged Off Loans

The `loan_status` column contains many categories like "Current", "Late", or "In Grace Period" that don’t reflect final outcomes. To create a meaningful binary classification problem, we filtered the dataset to keep only rows where `loan_status` is either:
- `"Fully Paid"` (loan repaid successfully)
- `"Charged Off"` (loan defaulted)

This simplifies the target variable and ensures clarity in modeling.


In [2]:
df['loan_status'].value_counts(dropna=False)

loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
NaN                                                         33
Name: count, dtype: int64

In [3]:
df = df[df['loan_status'].isin(['Fully Paid', 'Charged Off'])]

## Create a Binary Target Column (`default`)

We created a new column named `default` to serve as the target for classification:
- `1` for "Charged Off" loans (defaulted)
- `0` for "Fully Paid" loans (repaid)

This conversion allows us to model the problem as a binary classification task.


In [4]:
# Create the binary target
df['default'] = df['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)

# Check new label distribution
df['default'].value_counts(normalize=True)

default
0    0.800374
1    0.199626
Name: proportion, dtype: float64

## Column Removal: Preventing Data Leakage and Redundancy

To ensure our model is both realistic and robust, we carefully removed several columns from the dataset that fall into the following different categories:

In [5]:
list(df.columns)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 '

### 1. Leakage Columns
These columns contain information that would **not be available at the time of loan issuance**, such as payment history, recoveries, or post-loan credit scores. Including them would result in **data leakage**, leading to overly optimistic model performance. Examples include:
- Total payments and principal recovered (`total_pymnt`, `recoveries`)
- Last and next payment dates (`last_pymnt_d`, `next_pymnt_d`)
- Post-loan FICO scores (`last_fico_range_low`, `last_fico_range_high`)
- Hardship and debt settlement statuses

In [6]:
leakage_cols = [
    'loan_status',  # already used to create 'default'
    
    # Post-issuance payment/recovery info
    'out_prncp', 'out_prncp_inv',
    'total_pymnt', 'total_pymnt_inv',
    'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
    'recoveries', 'collection_recovery_fee',
    
    # Post-loan date fields
    'last_pymnt_d', 'last_pymnt_amnt',
    'next_pymnt_d', 'last_credit_pull_d',

    # Hardship-related (only happens if borrower struggles later)
    'hardship_flag', 'hardship_type', 'hardship_reason',
    'hardship_status', 'deferral_term', 'hardship_amount',
    'hardship_start_date', 'hardship_end_date',
    'payment_plan_start_date', 'hardship_length',
    'hardship_dpd', 'hardship_loan_status',
    'orig_projected_additional_accrued_interest',
    'hardship_payoff_balance_amount',
    'hardship_last_payment_amount',

    # Debt settlement information
    'debt_settlement_flag', 'debt_settlement_flag_date',
    'settlement_status', 'settlement_date',
    'settlement_amount', 'settlement_percentage',
    'settlement_term',

    # Post-issuance FICO scores
    'last_fico_range_low', 'last_fico_range_high'
]


### 2. Non-Predictive Metadata
These fields are either:
- Unique identifiers (`id`, `member_id`)
- Free-text or noisy fields (`emp_title`, `desc`)
- Redundant or uninformative (`zip_code`, `pymnt_plan`, `policy_code`)

These do not contribute meaningful signal and may increase noise or dimensionality.

In [7]:
non_predictive_cols = [
    'id', 'member_id',          # Unique identifiers
    'emp_title', 'url', 'desc', # Unstructured text fields
    'title',                    # Often duplicates purpose
    'zip_code', 'addr_state',   # Hard to generalize or engineer
    'issue_d',                  # Loan issuance date
    'pymnt_plan',               # Almost always 'n'
    'disbursement_method',      # Nearly always one value
    'policy_code'               # Constant = 1
]

### 3. Second Applicant Information
Our model does not distinguish between individual and joint applications. Therefore, we removed all features related to **secondary applicants** (`sec_app_*`), which are frequently missing and not applicable to most loans.

In [8]:
sec_app_cols = [
    'sec_app_fico_range_low', 'sec_app_fico_range_high',
    'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths',
    'sec_app_mort_acc', 'sec_app_open_acc',
    'sec_app_revol_util', 'sec_app_open_act_il',
    'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths',
    'sec_app_collections_12_mths_ex_med',
    'sec_app_mths_since_last_major_derog'
]

We combined all of the above into one list and removed them from the dataset using the following code:

In [9]:
# Combine both drop lists
cols_to_drop = leakage_cols + non_predictive_cols + sec_app_cols

# Drop
df = df.drop(columns=cols_to_drop, errors='ignore')


The remaining columns are:

In [10]:
list(df.columns)

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'purpose',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 'open_act_il',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_am

## Handling Missing Values

Dealing with missing data is a critical step in preparing the dataset for machine learning. In this section, we assess and handle missing values to ensure a clean, model-ready dataset.

### Step 1: Identify Columns with Missing Values

We begin by calculating the percentage of missing values in each column, sorted in descending order. This gives us insight into which columns have the most missing data.

In [11]:
# Find % of missing values for each column
missing = df.isnull().mean().sort_values(ascending=False)
missing.head(10)

revol_bal_joint                   0.986153
verification_status_joint         0.980975
dti_joint                         0.980824
annual_inc_joint                  0.980822
mths_since_last_record            0.830110
mths_since_recent_bc_dlq          0.762865
mths_since_last_major_derog       0.737049
mths_since_recent_revol_delinq    0.665533
il_util                           0.654343
mths_since_rcnt_il                0.610958
dtype: float64

### Step 2: Drop Columns with >30% Missing Values

Features with more than 30% missing values are dropped. These columns are likely too incomplete to provide reliable signal and may introduce noise if imputed.

In [12]:
threshold = 0.3
cols_to_drop = missing[missing > threshold].index
df = df.drop(columns=cols_to_drop)
print(cols_to_drop)

Index(['revol_bal_joint', 'verification_status_joint', 'dti_joint',
       'annual_inc_joint', 'mths_since_last_record',
       'mths_since_recent_bc_dlq', 'mths_since_last_major_derog',
       'mths_since_recent_revol_delinq', 'il_util', 'mths_since_rcnt_il',
       'all_util', 'total_cu_tl', 'open_acc_6m', 'inq_last_12m', 'inq_fi',
       'open_rv_24m', 'open_rv_12m', 'total_bal_il', 'open_il_24m',
       'open_il_12m', 'open_act_il', 'max_bal_bc', 'mths_since_last_delinq'],
      dtype='object')


### Step 3: Fill Missing Numeric Values with Median

For numeric features, we impute missing values using the median. The median is robust to outliers, which makes it suitable for skewed financial variables.


In [13]:
# Fill in missing values
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

### Step 4: Fill Missing Categorical Values with Mode

For categorical features, we impute missing values using the most frequent value (mode) to preserve the dominant category and ensure one-hot encoding works correctly.


In [14]:
categorical_cols = df.select_dtypes(include=['object']).columns
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

### Step 5: Final Check for Missing Values

We now verify that all missing values have been handled. The result should be zero.

In [15]:
df.isnull().sum().sum()

np.int64(0)

## Encoding Categorical Features

Most machine learning models require all input features to be numeric. However, many of the features in our dataset are categorical, such as `home_ownership`, `purpose`, or `verification_status`.

To make these categorical variables usable, we apply **one-hot encoding**, which transforms each category into a separate binary column (0 or 1). This allows the model to treat each category as an independent feature without introducing false ordinal relationships.

### Step 1: One-Hot Encode Categorical Variables

We use `pd.get_dummies()` to one-hot encode all non-numeric columns. The `drop_first=True` parameter avoids multicollinearity by dropping the first category from each feature.

In [16]:
# Find all non-numeric columns and one-hot encode them
df_encoded = pd.get_dummies(df, drop_first=True)


### Step 2: Verify All Features Are Now Numeric

After encoding, we check the data types of all columns to ensure everything is now numeric and ready for model training.


In [17]:
df_encoded.dtypes.value_counts()

bool       811
float64     56
int64        1
Name: count, dtype: int64

### Step 3: Final Check for Unencoded Categorical Columns

Even after one-hot encoding, it's good practice to confirm that no `object` (i.e., string-based) columns remain in the dataset. This step helps catch any features that were missed or unexpectedly formatted.

If the output is an empty list, it means all features are successfully numeric and the dataset is ready for scaling and modeling.

In [18]:
df_encoded.select_dtypes(include='object').columns

Index([], dtype='object')

## Splitting Features and Target

Before we can scale the data or train any models, we need to separate our dataset into:
- **Features (`X`)**: All the independent variables that the model will use to make predictions.
- **Target (`y`)**: The dependent variable — in this case, the `default` column, which indicates whether a borrower defaulted (1) or fully repaid (0) their loan.

This separation is a fundamental step in any supervised machine learning workflow.

In [19]:
X = df_encoded.drop('default', axis=1)  # All columns except the label
y = df_encoded['default']               # The label (0 or 1)
X.shape, y.shape

((1345310, 867), (1345310,))

## Exporting the Cleaned Dataset

After completing preprocessing — including missing value imputation, feature encoding, and column filtering — we save the cleaned dataset as a CSV file. This step ensures reproducibility and allows us to use the same preprocessed data across multiple notebooks and models without repeating the cleaning steps.

We exclude the index column from the saved file for clarity and compactness.

In [20]:
df_encoded.to_csv('../data/cleaned_credit_data.csv', index=False)